In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install -q keras

In [3]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.resnet50 import preprocess_input
import matplotlib.pyplot as plt

In [4]:
import os
os.chdir("/content/drive/My Drive/Images size 224")
!ls

CNNModels.ipynb  TRAIN_DIR
TEST_DIR	 vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


In [ ]:
#Training and testing directories
train = 'TRAIN_DIR/'
test = 'TEST_DIR/'

#Augumentation with only zoom and grayscale format

train_datagen2 = ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip = True,
                             zoom_range = 0.3
                            )

#Corresponding generator
train_generator2 = train_datagen2.flow_from_directory(
    train,
    color_mode='grayscale',
    batch_size=32,
    class_mode = 'categorical',
    target_size=(224,224),
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen2 = ImageDataGenerator()
#Using testing data as validation data

validation_generator2 = test_datagen2.flow_from_directory(
    test,
    color_mode='grayscale',
    batch_size=32,
    class_mode = 'categorical',
    target_size=(224,224),
    seed=42 )

x_train,y_train = next(train_generator2)
x_test,y_test = next(validation_generator2)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 224, 224, 1)
(32, 224, 224, 1)


In [ ]:
#Basic CNN Model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
import numpy as np
from keras.callbacks import ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),padding='same', activation='relu', input_shape = (224, 224, 1)))
model.add(MaxPool2D(pool_size=(1,1)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
#for fitting the model
from keras.callbacks import EarlyStopping

stop = EarlyStopping(
    monitor = 'val_accuracy',
    mode='max',
    patience= 8,
    verbose = 2,
    restore_best_weights = True
)

STEPS = train_generator2.samples//train_generator2.batch_size
VAL_STEPS = validation_generator2.samples//validation_generator2.batch_size
EPOCHS = 20

results = model.fit(
train_generator2,
steps_per_epoch=STEPS,
epochs=EPOCHS,
validation_data=validation_generator2,
validation_steps=VAL_STEPS,
callbacks=[stop]
)

Epoch 1/20
12/12 [==============================] - 128s 11s/step - loss: 10909.0448 - accuracy: 0.4353 - val_loss: 5039.5723 - val_accuracy: 0.2891
Epoch 2/20
12/12 [==============================] - 3s 228ms/step - loss: 2665.7018 - accuracy: 0.4885 - val_loss: 926.8793 - val_accuracy: 0.6172
Epoch 3/20
12/12 [==============================] - 3s 225ms/step - loss: 1583.3783 - accuracy: 0.4974 - val_loss: 411.9169 - val_accuracy: 0.5469
Epoch 4/20
12/12 [==============================] - 3s 227ms/step - loss: 537.3114 - accuracy: 0.5641 - val_loss: 149.0376 - val_accuracy: 0.6484
Epoch 5/20
12/12 [==============================] - 3s 229ms/step - loss: 235.9949 - accuracy: 0.6621 - val_loss: 205.5592 - val_accuracy: 0.5547
Epoch 6/20
12/12 [==============================] - 3s 230ms/step - loss: 254.4408 - accuracy: 0.6303 - val_loss: 122.1065 - val_accuracy: 0.6016
Epoch 7/20
12/12 [==============================] - 3s 226ms/step - loss: 246.4945 - accuracy: 0.5754 - val_loss: 107.4

**Starting With ResNet50**

In [ ]:
from keras import applications
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from keras.optimizers import SGD, Adam

In [ ]:
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (224,224,1))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation= 'softmax')(x)
resnetmodel = Model(inputs = base_model.input, outputs = predictions)
adam = Adam(lr=0.001)
resnetmodel.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
EPOCH = 30
stopping_resNet = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=5,
    verbose = 2,
    restore_best_weights = True
)

restNetModel = resnetmodel.fit(
    train_generator2,
    steps_per_epoch=STEPS,
    epochs=EPOCH,
    validation_data=validation_generator2,
    validation_steps=VAL_STEPS,
    callbacks=[stopping_resNet]
)

Epoch 1/30
12/12 [==============================] - 10s 452ms/step - loss: 2.3921 - accuracy: 0.5252 - val_loss: 72448.8516 - val_accuracy: 0.3203
Epoch 2/30
12/12 [==============================] - 5s 377ms/step - loss: 0.7494 - accuracy: 0.7259 - val_loss: 42700.0039 - val_accuracy: 0.2891
Epoch 3/30
12/12 [==============================] - 5s 382ms/step - loss: 0.5526 - accuracy: 0.7918 - val_loss: 7603.9463 - val_accuracy: 0.2891
Epoch 4/30
12/12 [==============================] - 5s 385ms/step - loss: 0.5544 - accuracy: 0.8513 - val_loss: 618.9281 - val_accuracy: 0.3281
Epoch 5/30
12/12 [==============================] - 5s 391ms/step - loss: 0.4151 - accuracy: 0.8594 - val_loss: 354.8542 - val_accuracy: 0.3203
Epoch 6/30
12/12 [==============================] - 5s 395ms/step - loss: 0.2853 - accuracy: 0.8763 - val_loss: 23.8628 - val_accuracy: 0.3516
Epoch 7/30
12/12 [==============================] - 5s 399ms/step - loss: 0.3428 - accuracy: 0.8686 - val_loss: 3.6980 - val_accura

**CNN with Batch Normalization**

In [ ]:
model2 = Sequential()

#1-conv layer
model2.add(Conv2D(64, (3,3), padding = 'same', input_shape=(256, 256,1)))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))
model2.add(Dropout(0.25))


#2-conv layer
model2.add(Conv2D(128,(5,5),padding='same'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))
model2.add(Dropout(0.25))



#Flatten
model2.add(Flatten())


model2.add(Dense(256))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))

model2.add(Dense(512))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))


#output layer
model2.add(Dense(3,activation='softmax'))

opt = Adam(lr = 0.0001)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics = ['accuracy'])

model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
batch_normalization_4 (Batch (None, 256, 256, 64)      256       
_________________________________________________________________
activation_4 (Activation)    (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     204928    
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 128, 128)    

In [ ]:
results_3layer = model2.fit(
    x=train_generator2,
    steps_per_epoch = STEPS,
    epochs = EPOCHS,
    validation_data = validation_generator2,
    validation_steps = VAL_STEPS,
    callbacks=[stop]
)

Epoch 1/20
12/12 [==============================] - 10s 591ms/step - loss: 0.8955 - accuracy: 0.6313 - val_loss: 1.0799 - val_accuracy: 0.5703
Epoch 2/20
12/12 [==============================] - 6s 453ms/step - loss: 0.5186 - accuracy: 0.8337 - val_loss: 1.3319 - val_accuracy: 0.2969
Epoch 3/20
12/12 [==============================] - 6s 450ms/step - loss: 0.4471 - accuracy: 0.8140 - val_loss: 1.4801 - val_accuracy: 0.3047
Epoch 4/20
12/12 [==============================] - 6s 449ms/step - loss: 0.3525 - accuracy: 0.8681 - val_loss: 1.5434 - val_accuracy: 0.4531
Epoch 5/20
12/12 [==============================] - 6s 452ms/step - loss: 0.3055 - accuracy: 0.8925 - val_loss: 1.6299 - val_accuracy: 0.6875
Epoch 6/20
12/12 [==============================] - 6s 453ms/step - loss: 0.3338 - accuracy: 0.8647 - val_loss: 1.8036 - val_accuracy: 0.6641
Epoch 7/20
12/12 [==============================] - 6s 451ms/step - loss: 0.2477 - accuracy: 0.9285 - val_loss: 1.7144 - val_accuracy: 0.6172
Epoch

**VGG16**

In [ ]:
#For RGB images

train_datagen = ImageDataGenerator(horizontal_flip=True, 
                             zoom_range = 0.3,
                            )

#Corresponding generator
train_generator = train_datagen.flow_from_directory(
    train,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (224,224),
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen = ImageDataGenerator()
#Using testing data as validation data

validation_generator = test_datagen.flow_from_directory(
    test,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (224,224),
    seed=42 )

x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 224, 224, 3)
(32, 224, 224, 3)


In [ ]:
from keras.applications.vgg16 import VGG16
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping

vgg16_weight_path = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = VGG16(
    weights=vgg16_weight_path,
    include_top=False, 
    input_shape=(224,224,3,)
)

NUM_CLASSES = 3

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

model.layers[0].trainable = False

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 75267     
Total params: 14,789,955
Trainable params: 75,267
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
EPOCHS_VGG = 30

VGG_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=10,
    verbose = 2,
    restore_best_weights = True
)

history = model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_VGG,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[VGG_STOP]
)

Epoch 1/30
12/12 [==============================] - 18s 938ms/step - loss: 11.1947 - accuracy: 0.4851 - val_loss: 0.8758 - val_accuracy: 0.8594
Epoch 2/30
12/12 [==============================] - 6s 487ms/step - loss: 1.2551 - accuracy: 0.9051 - val_loss: 0.6912 - val_accuracy: 0.9375
Epoch 3/30
12/12 [==============================] - 6s 477ms/step - loss: 1.3749 - accuracy: 0.9253 - val_loss: 1.0198 - val_accuracy: 0.9062
Epoch 4/30
12/12 [==============================] - 6s 476ms/step - loss: 1.2393 - accuracy: 0.9293 - val_loss: 1.4347 - val_accuracy: 0.8750
Epoch 5/30
12/12 [==============================] - 6s 474ms/step - loss: 0.8202 - accuracy: 0.9292 - val_loss: 0.4654 - val_accuracy: 0.9297
Epoch 6/30
12/12 [==============================] - 6s 463ms/step - loss: 0.4313 - accuracy: 0.9684 - val_loss: 0.4197 - val_accuracy: 0.9531
Epoch 7/30
12/12 [==============================] - 6s 460ms/step - loss: 0.6270 - accuracy: 0.9536 - val_loss: 0.5970 - val_accuracy: 0.9297
Epoc

**MobileNet V2**

In [31]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop

In [48]:
#Training and testing directories
train = 'TRAIN_DIR/'
test = 'TEST_DIR/'

#For RGB images
train_datagen = ImageDataGenerator(horizontal_flip=True, 
                             zoom_range = 0.3,
                             preprocessing_function = preprocess_input)

#Corresponding generator
train_generator = train_datagen.flow_from_directory(
    train,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (224,224),
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input
)
#Using testing data as validation data

validation_generator = test_datagen.flow_from_directory(
    test,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (224,224),
    seed=42 )

x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 224, 224, 3)
(32, 224, 224, 3)


In [50]:
mobilenet_weight_path = 'mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'

mobileNet = MobileNetV2(
    input_shape = (224,224,3),
    alpha=1.0,
    include_top=False,
    weights= mobilenet_weight_path
)

MN_model = Sequential()
MN_model.add(mobileNet)
MN_model.add(layers.GlobalAveragePooling2D())
MN_model.add(layers.Dropout(0.5))
MN_model.add(layers.Dense(3, activation='softmax'))
    
MN_model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.00002),
        metrics=['accuracy']
    )

MN_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_6 ( (None, 1280)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 3843      
Total params: 2,261,827
Trainable params: 2,227,715
Non-trainable params: 34,112
_________________________________________________________________


In [52]:
#for fitting the model
from keras.callbacks import EarlyStopping

EPOCHS_MN = 30

STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size

MN_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=7,
    verbose = 2,
    restore_best_weights = True
)

history = MN_model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_MN,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[MN_STOP]
)

Epoch 1/30
12/12 [==============================] - 7s 555ms/step - loss: 0.0742 - accuracy: 0.9894 - val_loss: 0.8846 - val_accuracy: 0.7109
Epoch 2/30
12/12 [==============================] - 7s 543ms/step - loss: 0.0720 - accuracy: 0.9788 - val_loss: 0.9429 - val_accuracy: 0.6875
Epoch 3/30
12/12 [==============================] - 7s 549ms/step - loss: 0.0631 - accuracy: 0.9815 - val_loss: 0.9499 - val_accuracy: 0.6875
Epoch 4/30
12/12 [==============================] - 7s 547ms/step - loss: 0.0648 - accuracy: 0.9815 - val_loss: 0.9233 - val_accuracy: 0.6953
Epoch 5/30
12/12 [==============================] - 7s 546ms/step - loss: 0.0529 - accuracy: 0.9841 - val_loss: 0.9042 - val_accuracy: 0.6953
Epoch 6/30
12/12 [==============================] - 7s 542ms/step - loss: 0.0593 - accuracy: 0.9841 - val_loss: 0.8862 - val_accuracy: 0.7188
Epoch 7/30
12/12 [==============================] - 7s 553ms/step - loss: 0.0483 - accuracy: 0.9947 - val_loss: 0.8184 - val_accuracy: 0.7422
Epoch 

**SqueezeNet**

In [57]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


In [59]:
bnmomemtum=0.9
def fire(x, squeeze, expand):
  y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
  y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
  y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
  y1 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y1)
  y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
  y3 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y3)
  return tf.keras.layers.concatenate([y1, y3])

def fire_module(squeeze, expand):
  return lambda x: fire(x, squeeze, expand)

x = tf.keras.layers.Input(shape=[224,224, 3]) # input is 192x192 pixels RGB

y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(64, 128)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.GlobalAveragePooling2D()(y)
y = tf.keras.layers.Dense(3, activation='softmax')(y)

model = tf.keras.Model(x, y)
model.compile(
    optimizer='adam',
    loss= 'categorical_crossentropy',
    metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 224, 224, 32) 896         input_13[0][0]                   
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 224, 224, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 24) 792         batch_normalization[0][0]        
______________________________________________________________________________________________

In [60]:
EPOCHS_SQ = 30

STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size

SQ_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=7,
    verbose = 2,
    restore_best_weights = True
)

history = MN_model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_SQ,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[SQ_STOP]
)

Epoch 1/30
12/12 [==============================] - 7s 555ms/step - loss: 0.0174 - accuracy: 0.9947 - val_loss: 0.2042 - val_accuracy: 0.9141
Epoch 2/30
12/12 [==============================] - 7s 549ms/step - loss: 0.0128 - accuracy: 1.0000 - val_loss: 0.2175 - val_accuracy: 0.9062
Epoch 3/30
12/12 [==============================] - 7s 546ms/step - loss: 0.0133 - accuracy: 1.0000 - val_loss: 0.1877 - val_accuracy: 0.9141
Epoch 4/30
12/12 [==============================] - 7s 555ms/step - loss: 0.0178 - accuracy: 0.9921 - val_loss: 0.1763 - val_accuracy: 0.9219
Epoch 5/30
12/12 [==============================] - 7s 546ms/step - loss: 0.0143 - accuracy: 1.0000 - val_loss: 0.1702 - val_accuracy: 0.9219
Epoch 6/30
12/12 [==============================] - 7s 539ms/step - loss: 0.0093 - accuracy: 1.0000 - val_loss: 0.1564 - val_accuracy: 0.9219
Epoch 7/30
12/12 [==============================] - 7s 548ms/step - loss: 0.0167 - accuracy: 0.9974 - val_loss: 0.1408 - val_accuracy: 0.9453
Epoch 